# IL HW3
## Part 2 | MAZE
Ali Saeizadh
810196477

In [324]:
from amalearn.reward import RewardBase
from amalearn.agent import AgentBase


In [325]:
from amalearn.environment import EnvironmentBase
import gym
import pandas as pd
import numpy as np
from random import randint
from pprint import pprint


In [326]:
map_df = pd.read_csv('maze.map', header=None)
map = map_df.to_numpy()
print(map)


[[2 1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1 1]]


In [327]:
class Environment(EnvironmentBase):
    def __init__(self, obstacle=[], id=0, action_count=9, actionPrice=-0.01, goalReward=1000, punish=-1, j_limit=15, i_limit=15, p=0.8, container=None):
        """
        initialize your variables
        """
        map_df = pd.read_csv('maze.map', header=None)
        self.map = map_df.to_numpy()
        self.actions_name = np.array(
            ['up', 'down', 'left', 'right', 'upright', 'upleft', 'downleft', 'dwonright', 'remain'])
        self.actions = np.array([np.array([0, 1]), np.array([0, -1]), np.array([-1, 0]), np.array(
            [1, 0]), np.array([1, 1]), np.array([-1, 1]), np.array([-1, -1]), np.array([1, -1]), np.array([0, 0])])
        self.A = list(range(0, self.actions.shape[0]))
        self.S = list(range(0, self.map.size))
        self.i_limit = 15
        self.j_limit = 15
        self.p = p
        self.goal_reward = goalReward
        self.punish = punish
        self.action_price = actionPrice
        self.current_action = 0
        self.current_state = np.array([14, 14])
        self.agent_action_rec = []
        self.agent_states = [self.current_state]

    def isStatePossible(self, state):
        """if given state is possible (not out of the grid and not obstacle) return ture"""
        if(state[0] >= self.i_limit or state[0] < 0):
            return False
        if(state[1] >= self.j_limit or state[1] < 0):
            return False
        if(map[state[0], state[1]] == 0):
            return False
        return True

    def isAccessible(self, state, state_p):
        """if given state is Accesible (we can reach state_p by doing an action from state) return true"""
        if(self.isStatePossible(state_p) == False):
            return False
        if (np.linalg.norm(state - state_p) - np.sqrt(2) > 0):
            return False
        return True

    def is_obstacle(self, state):
        if(map[state[0], state[1]] == 0):
            return True
        return False

    def number_of_accessible(self, state):
        num = 0
        for i in range(len(self.actions)):
            if(self.isStatePossible(state + self.actions[i])):
                num += 1
        return num

    def getTransitionStatesAndProbs(self, state, action, state_p):
        """return probability of transition or T(sp,a,s)"""
        if(self.isAccessible(state, state_p) == False):
            return 0
        if(np.array_equal(state_p - state, self.actions[action])):
            return self.p
        else:
            return (1 - self.p) / self.number_of_accessible(state)

    def getReward(self, state, action, state_p):
        """return reward of transition"""
        if(self.isAccessible(state, state_p) == False):
            return self.punish
        if(self.map[state_p[0], state_p[1]] == 2):
            return self.goal_reward + self.action_price
        if(self.is_obstacle(state_p)):
            return self.punish + self.action_price
        return self.action_price

    def sample_all_rewards(self):
        return

    def calculate_reward(self, action):
        # replaced by getReward
        return

    def terminated(self):
        S = self.available_states()
        if(np.array_equal(self.current_state, np.array([0, 0]))):
            return True
        return False

    def observe(self):
        return {}

    def available_actions(self):
        return self.actions

    def possible_actions(self, state):
        poss_actions = np.zeros(len(self.A))
        for a in self.A:
            state_p = state + self.actions[a]
            if(self.isStatePossible(state_p)):
                poss_actions[a] = 1
        return poss_actions

    def possible_states(self, state):
        poss_states = np.zeros(len(self.S))
        all_states = self.available_states()
        for s in self.S:
            if(self.isAccessible(state, all_states[s])):
                poss_states[s] = 1
        return poss_states

    def available_states(self):
        states = [np.array([x, y]) for x in range(self.map.shape[0])
                  for y in range(self.map.shape[1])]
        counter = list(range(0, len(states)))
        S = dict(zip(counter, states))
        return S

    def next_state(self, action):
        choise = np.random.uniform(0, 1)
        error_action = self.A.copy()
        error_action.remove(action)
        if(choise > self.p):
            action = np.random.choice(error_action)

        state_p = self.current_state + self.actions[action]
        if(self.isAccessible(self.current_state, state_p) == False):
            self.agent_states.append(self.current_state)
            return [self.current_state, action]
        if(self.is_obstacle(state_p)):
            self.agent_states.append(self.current_state)
            return [self.current_state, action]
        self.current_state = state_p
        self.agent_states.append(self.current_state)
        return [self.current_state, action]

    def reset(self):
        self.current_state = 0

    def render(self, mode='human'):
        # print('{}:\taction={}'.format(self.state['length'], self.state['last_action']))
        return

    def step(self, action):
        # err_msg = "%r (%s) invalid" % (action, type(action))
        # assert self.action_space.contains(action), err_msg
        # print(self.current_state)
        info = self.get_info(action)
        [self.current_state, real_action] = self.next_state(action)
        reward = self.getReward(
            self.agent_states[-2], real_action, self.agent_states[-1])
        done = self.terminated()
        observation = self.observe()
        return observation, reward, done, info, self.current_state

    def close(self):
        return


In [328]:
class Agent(AgentBase):
    def __init__(self, id, environment, discount, theta):
        # initialize a random policy and V(s) = 0 for each state
        self.environment = environment
        self.mapp = {}
        self.Q = {}
        self.S = self.environment.available_states()
        self.A = list(range(0, self.environment.available_actions().shape[0]))
        # mapp states to its ids

        self.V = {s: 0 for s in range(0, len(self.S))}
        # init V
        self.policy = {s: 0 for s in range(0, len(self.S))}
        # init policy
        super(Agent, self).__init__(id, environment)
        self.discount = discount
        self.theta = theta
        self.current_state = len(self.S) - 1
        self.action_rec = []
        self.reward_rec = []

    def policy_evaluation(self):
        while True:
            delta = 0
            for s in self.S:
                old_V = self.V[s]
                a = self.policy[s]
                V_s = np.zeros(len(self.S))
                poss_states = self.environment.possible_states(self.S[s])
                for s_next in self.S:
                    if(poss_states[s_next] == 1):
                        V_s[s_next] = self.environment.getTransitionStatesAndProbs(self.S[s], a, self.S[s_next]) * (
                            self.environment.getReward(self.S[s], a, self.S[s_next]) + self.discount * self.V[s_next])
                self.V[s] = np.sum(V_s)
                delta = max(delta, abs(old_V - self.V[s]))
            if(delta < self.theta):
                break

    def policy_improvement(self):
        policy_stable = True
        for s in self.S:
            old_action = self.policy[s]
            action_val = np.zeros(len(self.A))
            poss_states = self.environment.possible_states(self.S[s])
            for a in self.A:
                V_s = np.zeros(len(self.S))
                for s_next in self.S:
                    if(poss_states[s_next] == 1):
                        V_s[s_next] = self.environment.getTransitionStatesAndProbs(self.S[s], a, self.S[s_next]) * (
                            self.environment.getReward(self.S[s], a, self.S[s_next]) + self.discount * self.V[s_next])
                action_val[a] = np.sum(V_s)
            action_val = (
                (-100000 * (1 - self.environment.possible_actions(self.S[s])))) + action_val
            self.policy[s] = np.argmax(action_val)
            if(old_action != self.policy[s]):
                policy_stable = False
        return policy_stable

    def value_iteration(self):
        while True:
            delta = 0
            for s in self.S:
                action_val = np.zeros(len(self.A))
                old_v = self.V[s]
                poss_states = self.environment.possible_states(self.S[s])
                for a in self.A:
                    V_s = np.zeros(len(self.S))
                    for s_next in self.S:
                        if(poss_states[s_next] == 1):
                            V_s[s_next] = self.environment.getTransitionStatesAndProbs(self.S[s], a, self.S[s_next]) * (
                                self.environment.getReward(self.S[s], a, self.S[s_next]) + self.discount * self.V[s_next])
                    action_val[a] = np.sum(V_s)
                action_val = (
                    (-100000 * (1 - self.environment.possible_actions(self.S[s])))) + action_val
                self.V[s] = np.max(action_val)
                delta = max(delta, abs(old_v - self.V[s]))
            if(delta < self.theta):
                break

        for s in self.S:
            action_val = np.zeros(len(self.A))
            poss_states = self.environment.possible_states(self.S[s])
            for a in self.A:
                V_s = np.zeros(len(self.S))
                for s_next in self.S:
                    if(poss_states[s_next] == 1):
                        V_s[s_next] = self.environment.getTransitionStatesAndProbs(self.S[s], a, self.S[s_next]) * (
                            self.environment.getReward(self.S[s], a, self.S[s_next]) + self.discount * self.V[s_next])
                action_val[a] = np.sum(V_s)
            action_val = (
                (-100000 * (1 - self.environment.possible_actions(self.S[s])))) + action_val
            self.policy[s] = np.argmax(action_val)

    def policy_iteration(self):
        while True:
            self.policy_evaluation()
            policy_stable = self.policy_improvement()
            if(policy_stable):
                break

    def get_key(self, my_dict, val):
        for key, value in my_dict.items():
            if np.array_equal(val, value):
                return key

    # def take_action(self):
    #     if(len(self.action_rec) < 1):
    #         self.value_iteration()
    #     action = self.policy[self.current_state]
    #     self.action_rec.append(action)
    #     obs, r, d, i, s = self.environment.step(action)
    #     self.reward_rec.append(r)
    #     self.environment.render()
    #     self.current_state = self.get_key(self.S, s)
    #     print(s)
    #     return obs, r, d, i


In [329]:
class Agent_Value_Iteration(Agent):
    def take_action(self):
        if(len(self.action_rec) < 1):
            self.policy_iteration()
        action = self.policy[self.current_state]
        self.action_rec.append(action)
        obs, r, d, i, s = self.environment.step(action)
        self.reward_rec.append(r)
        self.environment.render()
        self.current_state = self.get_key(self.S, s)
        return obs, r, d, i, s


class Agent_Policy_Iteration(Agent):
    def take_action(self):
        if(len(self.action_rec) < 1):
            self.value_iteration()
        action = self.policy[self.current_state]
        self.action_rec.append(action)
        obs, r, d, i, s = self.environment.step(action)
        self.reward_rec.append(r)
        self.environment.render()
        self.current_state = self.get_key(self.S, s)
        return obs, r, d, i, s


In [330]:
trial = 100
discount = 0.9
theta = 0.1
env = Environment()
no_friction_env = Environment(obstacle=[], id=1, action_count=9, actionPrice=0,
                              goalReward=1000, punish=-0.01, j_limit=15, i_limit=15, p=0.8, container=None)
high_friction_env = Environment(obstacle=[], id=2, action_count=9, actionPrice=-1,
                                goalReward=100, punish=-10, j_limit=15, i_limit=15, p=0.8, container=None)
agent_policy_original = Agent_Policy_Iteration(1, env, discount, theta)
agent_policy_no_friction = Agent_Policy_Iteration(
    2, no_friction_env, discount, theta)
agent_policy_high_friction = Agent_Policy_Iteration(
    3, high_friction_env, discount, theta)
agent_value = Agent_Value_Iteration(4, high_friction_env, discount, theta)
rewrads = np.zeros([4, trial])
trajectory = np.zeros([4, trial, 2])
for t in range(trial):
    obs, rewrads[0, t], d, i, trajectory[0,
                                         t] = agent_policy_original.take_action()
for t in range(trial):
    obs, rewrads[1, t], d, i, trajectory[1,
                                         t] = agent_policy_no_friction.take_action()
for t in range(trial):
    obs, rewrads[2, t], d, i, trajectory[2,
                                         t] = agent_policy_high_friction.take_action()
for t in range(trial):
    obs, rewrads[3, t], d, i, trajectory[3, t] = agent_value.take_action()


NameError: name 'rewards' is not defined